In [16]:
import google.generativeai as genai
import configparser
import json
import re

In [2]:
# leer archivo config.ini
config = configparser.ConfigParser()
config.read('config.ini')
GOOGLE_API_KEY = config['GOOGLE']['GOOGLE_API_KEY']







In [3]:
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [4]:
context='Eres un chabot'
query='¿Cómo te llamas?'
response = model.generate_content(f"Contexto: {context}\n\nPregunta: {query}")
print(response.text)

No tengo nombre.  Soy un modelo de lenguaje grande.



In [23]:
consulta = """
"Buenas tardes, 
al intentar iniciar el tramite de renovación de habilitación Municipal sin modificaciones el sistema nos devuelve el siguiente mensaje:
""Advertencia
Deberás realizar el Trámite de Renovación con modificaciones - Modificar Actividad porque hemos detectado diferencias entre la/s actividad/es y el/los destino/s del último certificado otorgado.""
como debemos proceder para renovar la habilitación y cuales son las diferencias detectadas?
Gracias
Saludos
"""

prompt = f"""
Determine cinco temas que se están tratando en el siguiente texto, que está delimitado por tres comillas invertidas.
Haga que cada elemento tenga una o dos palabras.
Formatee su respuesta como una lista de elementos separados por comas.
Indique el sentimiento general del texto.
Indique si el remitente está expresando enojo.
Texto: '''{consulta}'''
Salida en formato JSON:
"Temas": 
"Sentimiento": 
"Enojo": "Si/No"
"""
response = model.generate_content(prompt)
print(response.text)

```json
{
  "Temas": "Renovación, Habilitación, Municipal, Sistema, Diferencias",
  "Sentimiento": "Frustración",
  "Enojo": "No"
}
```



Recorrer la lista de consultas de habilitaciones y obtener tematicas claves y sentimientos de las mismas


In [5]:
import pandas as pd

data = pd.read_excel('/home/gcasado0/proyectos/chatbot/Consultas_habilitaciones.xlsx')
data.head()

,Serial number,Submission ID,Submission URI,Creado,La consulta es,Domicilio: Dirección del consultante,Domicilio: ubi_o_outRos,Domicilio: Piso,Domicilio: Dpto,Domicilio: id_calle,...,Número de: establecimiento o certficado de hablitación o TGI,Contenido consulta,codigo_gestion,PDF,cuit,nivel,mail,access_token,cod_tramite,Análisis de la rta
0,499,1794755,/node/1754?check_logged_in=1,2024-01-03 09:26:32,Para una tercera persona,NUEVE DE JULIO y RODRIGUEZ MARTIN,NaN,NaN,NaN,72000.0,...,3185320,- la actividad declarada 829100 corresponde a ...,3423-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,27147589943,,alepedrido@hotmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,Lo enviaron al CMD dado que con el nro. de est...
1,500,1798854,/node/1754,2024-01-04 09:13:08,Para mí,CORDOBA 1000,NaN,NaN,NaN,43000.0,...,NaN,"Buenos días. Quisiera consultar ""Qué tipo de h...",5585-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,27373328451,,jumadacunhakessler@gmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN
2,501,1800249,/node/1754?check_logged_in=1,2024-01-04 13:40:34,Para mí,SARMIENTO DOMINGO FAUSTINO 1242,NaN,1242,NaN,87300.0,...,NaN,quiero saber como se realiza una habilitacion ...,[webform:handler:remote_post_pic:completed:dat...,https://gestorpic.rosario.gob.ar/inicio/webfor...,27265509180,3,sanysilva@icloud.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN
3,502,1804219,/node/1754,2024-01-05 15:00:21,Para mí,SAN JUAN 3011,NaN,NaN,NaN,85650.0,...,NaN,Hola quiero consultar ya que quiero realizar u...,8536-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,27426087443,2,sianuu.robledo@gmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN
4,503,1809440,/node/1754?check_logged_in=1,2024-01-08 08:57:03,Para mí,NECOCHEA GRAL. MARIANO 3453,NaN,NaN,NaN,71000.0,...,NaN,Para hacerme el monotributo me pide el contado...,11972-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,20302544124,2,chukycuervosl@hotmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN


In [9]:
consulta = data['Contenido consulta'][0]
consulta

'- la actividad declarada 829100 corresponde a la de un agente plus pago, santa fe servicios, pago facil, etc?. Si no es la correcta cual seria. \n- se quiere anexar un polirrubro, de venta de golosinas, cigarrillos, etc. Cual seria el codigo de actividad ya que el 471190 no discrimina entre vta de tabaco, alimentos, regaleria, etc y creo que todos tienen alicuotas distintas. El establecimiento ya esta inscripto en el regimen general de DREI. Cuales serian los codigos de actividad a anexar.\nmuchas gracias.\n'

In [11]:
prompt = f"""
Extraer un resumen de lo que quiere el remitente en la siguiente consulta, que está delimitada por tres comillas invertidas.
Extraer los temas principales de la consulta.
Haga que cada elemento tenga una o dos palabras.Formatee su respuesta como una lista de elementos separados por comas.
Indique el sentimiento general del texto.
Indique si el remitente está expresando enojo.

Texto: '''{consulta}'''
Salida en formato JSON:
"Resumen":
"Temas": 
"Sentimientos": 
"Enojo": "Si/No"
"""
response = model.generate_content(prompt)
print(response.text)

```json
{
  "Resumen": "Confirmación código actividad y anexión polirrubro.",
  "Temas": "Código actividad, polirrubro, golosinas, cigarrillos, alicuotas.",
  "Sentimientos": "Preocupación, incertidumbre.",
  "Enojo": "No"
}
```



In [23]:
for i in range(15):
    consulta = data['Contenido consulta'][i]
    prompt = f"""
    Extraer un resumen de lo que quiere el remitente en la siguiente consulta, que está delimitada por tres comillas invertidas.
    Extraer los temas principales de la consulta.
    Haga que cada elemento tenga una o dos palabras. Formatee su respuesta como una lista de elementos separados por comas.
    Indique el sentimiento general del texto.
    Indique si el remitente está expresando enojo.
    Indique si la consulta es simple de resolver o no.
    Indique si el remitente no sabe que actividad elegir para su habilitacion.

    Texto: '''{consulta}'''
    Salida en formato JSON:
    "Resumen":
    "Temas": 
    "Sentimientos": 
    "Enojo": "Si/No"
    "Complejidad": "Simple/Compleja"
    "No sabe actividad": "Si/No"
    """
    response = model.generate_content(prompt)
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Resumen'] = analisis['Resumen']
            data.loc[i, 'Temas'] = analisis['Temas']
            data.loc[i, 'Complejidad'] = analisis['Complejidad']
            data.loc[i, 'Sentimientos'] = analisis['Sentimientos']
            data.loc[i, 'Enojo'] = analisis['Enojo']
            # No sabe actividad
            data.loc[i, 'No sabe actividad'] = analisis['No sabe actividad']

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")

# extraer del dataframe solo las siguientes columnas: 'Contenido consulta', 'Resumen', 'Temas', 'Sentimientos', 'Enojo'
data_analizado = data[['Temática ','Contenido consulta', 'Resumen', 'Temas','Complejidad', 'Sentimientos', 'Enojo','No sabe actividad']]

# guardar el dataframe data con las respuestas en un archivo Excel
data_analizado.to_excel('Consultas_habilitaciones_resumen.xlsx', index=False)


```json
{
  "Resumen": "Confirmación código actividad y anexión de polirrubro para venta de golosinas y cigarrillos.",
  "Temas": "Código actividad, polirrubro, golosinas, cigarrillos, alicuotas",
  "Sentimientos": "Preocupación, duda",
  "Enojo": "No",
  "Complejidad": "Compleja",
  "No sabe actividad": "Si"
}
```

```json
{
  "Resumen": "Habilitación cervecería sin comida",
  "Temas": "Habilitación, cervecería, snacks",
  "Sentimientos": "Neutral, inquisitivo",
  "Enojo": "No",
  "Complejidad": "Compleja",
  "No sabe actividad": "Si"
}
```

```json
{
  "Resumen": "Información sobre habilitación de comercio de ropa.",
  "Temas": "Habilitación, comercio, ropa, pasos, costos",
  "Sentimientos": "Neutro, cortés",
  "Enojo": "No",
  "Complejidad": "Compleja",
  "No sabe actividad": "No"
}
```

```json
{
  "Resumen": "Necesita ayuda para determinar el rubro de habilitación para expendio de bebidas alcohólicas.",
  "Temas": "Habilitación, Bebidas alcohólicas, Rubro",
  "Sentimientos": "Conf